# Calculate the background per plate

In [2]:
import os
import re
import sys
import glob
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage.io import imread
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm

sys.path.append("../scripts")
from img_utils import letter_dict, channel_dict
letter_dict_rev = {v: k for k, v in letter_dict.items()}
channel_dict_rev = {v: k for k, v in channel_dict.items()}

In [11]:
plate_bg = pl.read_parquet("../outputs/1.plate_bg_summary/2025_01_27_Batch_13/plate_bg.parquet")
plate_bg.join(
    plate_bg.group_by("plate","channel").agg(pl.col("median").median().alias("plate_channel_median")), 
    on=["plate","channel"]
).filter((pl.col("channel")=="GFP")&(pl.col("perc_90")<pl.col("plate_channel_median"))).sort(pl.col("well"))

img_path,plate,median,perc_90,perc_95,perc_99,site,channel,well,plate_channel_median
str,str,f64,f64,f64,f64,str,str,str,f64
"""../inputs/images//2025_01_27_B…","""2025_01_27_B13A7A8P2_T1""",152.0,173.0,178.0,187.0,"""03""","""GFP""","""F24""",174.0
"""../inputs/images//2025_01_27_B…","""2025_01_27_B13A7A8P2_T3""",153.0,173.0,178.0,186.0,"""03""","""GFP""","""H24""",174.0
"""../inputs/images//2025_01_27_B…","""2025_01_27_B13A7A8P2_T4""",149.0,170.0,175.0,183.0,"""02""","""GFP""","""H24""",171.0
"""../inputs/images//2025_01_27_B…","""2025_01_27_B13A7A8P2_T3""",148.0,169.0,174.0,183.0,"""06""","""GFP""","""I03""",174.0
"""../inputs/images//2025_01_27_B…","""2025_01_27_B13A7A8P2_T2""",147.0,168.0,172.0,181.0,"""05""","""GFP""","""N18""",174.0
"""../inputs/images//2025_01_27_B…","""2025_01_27_B13A7A8P2_T2""",152.0,173.0,178.0,187.0,"""07""","""GFP""","""N24""",174.0


In [8]:
plate_bg.group_by("plate","channel").agg(pl.col("median").median().alias("plate_channel_median"))

plate,channel,plate_channel_median
str,str,f64
"""2025_01_27_B13A7A8P2_T4""","""DAPI""",190.0
"""2025_01_27_B13A7A8P2_T1""","""DAPI""",192.0
"""2025_01_27_B13A7A8P1_T1""","""AGP""",197.0
"""2025_01_27_B13A7A8P2_T1""","""GFP""",174.0
"""2025_01_27_B13A7A8P1_T2""","""Mito""",556.0
…,…,…
"""2025_01_27_B13A7A8P2_T2""","""AGP""",192.0
"""2025_01_27_B13A7A8P2_T3""","""AGP""",189.0
"""2025_01_27_B13A7A8P1_T3""","""DAPI""",187.0


In [ ]:
## tiff_image
tiff_image_path = "../inputs/images/2025_01_27_Batch_13/images/2025_01_27_B13A7A8P1_T1__2025_01_27T08_46_50_Measurement_1/Images/r01c01f01p01-ch1sk1fk1fl1.tiff"
def process_tiff_img(tiff_image_path):
    img = imread(tiff_image_path)
    # median = float(np.median(img))
    percentiles = np.percentile(img, q=np.array([50,90,95,99]))
    tiff_img_name = tiff_image_path.split("/")[-1]
    site = re.search(r"(?<=f)(\d{2})(?=p)", tiff_img_name.split('-')[0])[0]
    channel = channel_dict_rev[re.search(r"(?<=ch)(\d+)(?=sk)", tiff_img_name.split('-')[1])[0]]
    well_letter = letter_dict_rev[re.search(r'(?<=r)(\d{2})(?=c)', tiff_img_name.split('-')[0])[0]]
    well_num = re.search(r'(?<=c)(\d{2})(?=f)', tiff_img_name.split('-')[0])[0]
    well = f"{well_letter}{well_num}"

    return {"img_path": tiff_image_path, "plate": tiff_image_path.split('/')[-3].split("__")[0], 
            "median": percentiles[0], "perc_90": percentiles[1], "perc_95": percentiles[2], "perc_99": percentiles[3],
            "site": site, "channel": channel, "well": well}

# process_tiff_img(tiff_image_path)
# find all TIFFs
tiff_img_dir = "../inputs/images/"
paths = glob.glob(f"{tiff_img_dir}/*/images/*/Images/*.tiff", recursive=True)[:100]
records = []
# process in parallel with a progress bar
with ProcessPoolExecutor(max_workers=384) as exe:
    futures = {exe.submit(process_tiff_img, p): p for p in paths}
    for fut in tqdm(as_completed(futures), total=len(futures), desc="Processing TIFFs"):
        rec = fut.result()
        records.append(rec)

# # build DataFrame and write Parquet
# df = pl.DataFrame(records)

{'img_path': '../inputs/images/2025_01_27_Batch_13/images/2025_01_27_B13A7A8P1_T1__2025_01_27T08_46_50_Measurement_1/Images/r01c01f01p01-ch1sk1fk1fl1.tiff',
 'plate': '2025_01_27_B13A7A8P1_T1',
 'median': 188.0,
 'perc_90': 223.0,
 'perc_95': 235.0,
 'perc_99': 401.0,
 'site': '01',
 'channel': 'DAPI',
 'well': 'A01'}